In [1]:
!pip install gdown
!pip install pandarallel

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )


In [4]:
url='https://drive.google.com/drive/folders/1QvhirnOFoLqPZYeBz8M9oMZh-hxnOCnt?usp=sharing'

In [5]:
import gdown
gdown.download_folder(url, quiet=True)

['/kaggle/working/data_wav2vec2/benchmark.parquet',
 '/kaggle/working/data_wav2vec2/custom_data.parquet',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-43f0f09683d0ab82.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/cache-a99e762e72e1079e.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/data-00000-of-00001.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/test/dataset_info.json',
 '/kaggle/working/data_wav2vec2/data_sharded/test/state.json',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-bf7ee38c5d6c7cce.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/cache-c5e24c7182b4fbb4.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00000-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00001-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00002-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_sharded/train/data-00003-of-00005.arrow',
 '/kaggle/working/data_wav2vec2/data_shar

In [6]:
df_custom = pd.read_parquet('/kaggle/working/data_wav2vec2/custom_data.parquet')
df_wav2vec2_all = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_all.parquet')
df_wav2vec2_masked = pd.read_parquet('/kaggle/working/data_wav2vec2/wav2vec2_masked.parquet')

In [7]:
def mask_replace(sen,error_word):
    sen = sen.split()
    masked_index = sen.index("[MASK]")
    sen[masked_index] = error_word
    return " ".join(sen)

In [8]:
sen , error_word =  df_custom['masked_sen'][0], df_custom['error_word'][0]
print(sen)
print(error_word)

আর জামিনের [MASK] করলেন না অর্পিতা
সবেদন


In [9]:
df_custom['error_sentence'] = df_custom.parallel_apply(lambda x: mask_replace(x.masked_sen,x.error_word),axis=1)

In [10]:
df_custom = df_custom[['error_sentence','sentence']]
df_custom.rename(columns = {'error_sentence':'input','sentence':'output'},inplace=True)
df_custom

,input,output
0,আর জামিনের সবেদন করলেন না অর্পিতা,আর জামিনের আবেদন করলেন না অর্পিতা
1,অর্পিতা মুখার্জি কারাগ ক্ষ্ম রএই,অর্পিতা মুখার্জি কারাগারেই
2,তাঁর জী ন্ধ ন সংশয়ের আশঙ্কা রয়েছে,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খা র্ধ ি...,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ কর...
4,অর্পিতাকেও দিনের জেল হে গ্ম াজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন
...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোযে...,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোকে...
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আলোযে,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...
2992537,পঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে


In [11]:
df_wav2vec2_masked = df_wav2vec2_masked[['wav2vec2','sentence']]
df_wav2vec2_masked.rename(columns={'wav2vec2':'input','sentence':'output'},inplace=True)
df_wav2vec2_masked

,input,output
0,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,সার্ক দেশগুলোতে
2,শহজি জাদে,সহজেই যাওয়া যায়
3,শহজি জাদে,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে ...
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠ...,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠ...
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ...
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


In [12]:
df=df_custom.append(df_wav2vec2_masked)
df = df.dropna(how='any',axis=0) 
df

,input,output
0,আর জামিনের সবেদন করলেন না অর্পিতা,আর জামিনের আবেদন করলেন না অর্পিতা
1,অর্পিতা মুখার্জি কারাগ ক্ষ্ম রএই,অর্পিতা মুখার্জি কারাগারেই
2,তাঁর জী ন্ধ ন সংশয়ের আশঙ্কা রয়েছে,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খা র্ধ ি...,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ কর...
4,অর্পিতাকেও দিনের জেল হে গ্ম াজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে ...
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠ...,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠ...
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ...
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


In [13]:
df = df.sample(2041313)
df = df.reset_index(drop=True)
df

,input,output
0,অ্যান্ডারসনের এবার দলে ফেরার স্বপ্নটাও এখন রঙিন,অ্যান্ডারসনের আবার দলে ফেরার স্বপ্নটাও এখন রঙিন
1,আগস্টের হামলায় কেন এ অভিযোগ বিএনপির,আগস্টের হামলায় কেন এ অভিযোগ এল
2,স্থানীয় কৃষক মুজিবুর রহমান বধেন,স্থানীয় কৃষক মুজিবুর রহমান বলেন
3,নভোফেদোরিভকা বিমানঘাটউর ওপর হামলার ফলে বড় ধরনে...,নভোফেদোরিভকা বিমানঘাঁটির ওপর হামলার ফলে বড় ধরন...
4,আইএমএফ সালে কেনিয়ায় হস হ্ন রকনষেপ করে,আইএমএফ সালে কেনিয়ায় হস্তক্ষেপ করে
...,...,...
2041308,উপকরণ সাু কাপ,উপকরণ সাগু কাপ
2041309,গিয়ে দেখেন বাসের চসলক মফিজুর রহমান মারা গেছেন,গিয়ে দেখেন বাসের চালক মফিজুর রহমান মারা গেছেন
2041310,ভোগতী নরেন্দ্রপুর এলাকায় খেতের সরেজমিনে নষ্ট হ...,ভোগতী নরেন্দ্রপুর এলাকায় খেতের ধান নষ্ট হয়ে গেছে
2041311,ওভার শেষে কুমিল্ল ক্ষ্ণ উইকেটে,ওভার শেষে কুমিল্লা উইকেটে


In [14]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((1837181, 2), (204132, 2))

In [15]:
train_data = Dataset.from_pandas(train_df.sample(1837181))
test_data = Dataset.from_pandas(test_df)

In [16]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [17]:
max_len = 64 

In [18]:
model_name = 'flax-community/bengali-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
def prepare_data(example):
    example["input_ids"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).input_ids
    example["attention_mask"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).attention_mask
    example["labels"] = tokenizer(example["output"], padding="max_length", max_length=64).input_ids
    return example

In [20]:
train_data = train_data.map(prepare_data,remove_columns=train_data.column_names)

  0%|          | 0/1837181 [00:00<?, ?ex/s]

In [21]:
test_data = test_data.map(prepare_data)

  0%|          | 0/204132 [00:00<?, ?ex/s]

In [22]:
train_data.cleanup_cache_files()
test_data.cleanup_cache_files()

0

In [23]:
train_data.save_to_disk('train')
test_data.save_to_disk('test')

In [24]:
!rm -rf /kaggle/working/data_wav2vec2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
